
# Planificador de jornada del estudiante (A* y Branch & Bound)

Este cuaderno implementa el **modelo y los algoritmos** para el proyecto de *Algoritmos Avanzados de Búsqueda y Optimización*.



## 1) Definición del problema (resumen)

- Conjunto de actividades con utilidad, duración y ventanas de tiempo.
- Matriz de tiempos de viaje entre actividades.
- Objetivo: **maximizar** utilidad menos penalización por transporte `α`.



## 2) Clases de datos


In [ ]:

from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Optional, Set
import heapq

@dataclass(frozen=True)
class Activity:
    id: int
    name: str
    value: float
    duration: float
    open_time: float
    close_time: float

@dataclass
class Instance:
    activities: List[Activity]
    travel_time: List[List[float]]
    Tmax: float
    alpha: float = 0.0
    start_id: Optional[int] = None

    def __post_init__(self):
        n = len(self.activities)
        assert len(self.travel_time) == n and all(len(row) == n for row in self.travel_time)
        self.id_to_idx = {a.id: i for i, a in enumerate(self.activities)}

    def idx(self, act_id: int) -> int:
        return self.id_to_idx[act_id]

    def d(self, from_id: int, to_id: int) -> float:
        return self.travel_time[self.idx(from_id)][self.idx(to_id)]

    def activity_by_id(self, act_id: int) -> Activity:
        return self.activities[self.idx(act_id)]

@dataclass(order=True)
class State:
    priority: float
    current_id: int = field(compare=False)
    time: float = field(compare=False)
    collected_value: float = field(compare=False)
    travel_cost: float = field(compare=False)
    visited: frozenset = field(compare=False)
    route: Tuple[int, ...] = field(compare=False)

    def total_objective(self, alpha: float) -> float:
        return self.collected_value - alpha * self.travel_cost



## 3) Factibilidad y utilidades


In [ ]:

from typing import Optional, Tuple

def feasible_transition(inst: Instance, from_id: int, to_id: int, current_time: float) -> Optional[Tuple[float, float]]:
    act_j = inst.activity_by_id(to_id)
    travel = inst.d(from_id, to_id)
    arrive = current_time + travel
    start = max(arrive, act_j.open_time)
    if start > act_j.close_time:
        return None
    finish = start + act_j.duration
    if finish > inst.Tmax:
        return None
    return (start, finish)

def upper_bound_value(inst: Instance, state: State) -> float:
    remaining_time = inst.Tmax - state.time
    ub = state.collected_value
    remaining = [a for a in inst.activities if a.id not in state.visited]
    remaining.sort(key=lambda a: a.value / max(a.duration, 1e-9), reverse=True)
    t = 0.0
    for a in remaining:
        if t + a.duration <= remaining_time:
            ub += a.value
            t += a.duration
        else:
            if a.duration > 0 and remaining_time - t > 0:
                frac = (remaining_time - t) / a.duration
                ub += a.value * frac
            break
    return ub

def heuristic(inst: Instance, state: State) -> float:
    return 0.0



## 4) Algoritmo A*


In [ ]:

def _astar_from(inst: Instance, start_state: State) -> State:
    open_heap: List[State] = []
    heapq.heappush(open_heap, start_state)
    best: State = start_state
    seen: Dict[Tuple[int, frozenset], float] = {}

    while open_heap:
        s = heapq.heappop(open_heap)
        if s.total_objective(inst.alpha) > best.total_objective(inst.alpha):
            best = s
        for a in inst.activities:
            if a.id in s.visited:
                continue
            feas = feasible_transition(inst, s.current_id, a.id, s.time) if s.current_id != a.id else (s.time, s.time)
            if feas is None:
                continue
            start, finish = feas
            travel = 0.0 if s.current_id == a.id else inst.d(s.current_id, a.id)
            ns = State(priority=0.0,
                       current_id=a.id, time=finish,
                       collected_value=s.collected_value + a.value,
                       travel_cost=s.travel_cost + travel,
                       visited=s.visited | {a.id},
                       route=s.route + (a.id,))
            key = (ns.current_id, ns.visited)
            val = ns.total_objective(inst.alpha)
            if key in seen and seen[key] >= val:
                continue
            seen[key] = val
            ns.priority = -(val) + heuristic(inst, ns)
            heapq.heappush(open_heap, ns)
    return best

def astar_maximize(inst: Instance) -> State:
    if inst.start_id is None:
        best_state: Optional[State] = None
        for a in inst.activities:
            ok = feasible_transition(inst, a.id, a.id, 0.0)
            if ok is None:
                continue
            start, finish = ok
            s = State(priority=0.0, current_id=a.id, time=finish,
                      collected_value=a.value, travel_cost=0.0,
                      visited=frozenset({a.id}), route=(a.id,))
            s.priority = -(s.total_objective(inst.alpha)) + heuristic(inst, s)
            cand = _astar_from(inst, s)
            if (best_state is None) or (cand.total_objective(inst.alpha) > best_state.total_objective(inst.alpha)):
                best_state = cand
        if best_state is None:
            return State(priority=0.0, current_id=-1, time=0.0, collected_value=0.0, travel_cost=0.0,
                         visited=frozenset(), route=())
        return best_state
    else:
        initial = State(priority=0.0, current_id=inst.start_id, time=0.0, collected_value=0.0,
                        travel_cost=0.0, visited=frozenset(), route=())
        initial.priority = -(initial.total_objective(inst.alpha)) + heuristic(inst, initial)
        return _astar_from(inst, initial)



## 5) Branch & Bound


In [ ]:

def branch_and_bound(inst: Instance) -> State:
    best: Optional[State] = None
    def dfs(current_id: int, time: float, value: float, travel_cost: float, visited: Set[int], route: List[int]):
        nonlocal best
        current_state = State(priority=0.0, current_id=current_id, time=time, collected_value=value,
                              travel_cost=travel_cost, visited=frozenset(visited), route=tuple(route))
        if (best is None) or (current_state.total_objective(inst.alpha) > best.total_objective(inst.alpha)):
            best = current_state
        ub = upper_bound_value(inst, current_state)
        ub_obj = ub - inst.alpha * travel_cost
        if best is not None and ub_obj <= best.total_objective(inst.alpha) + 1e-9:
            return
        for a in inst.activities:
            if a.id in visited:
                continue
            feas = feasible_transition(inst, current_id, a.id, time)
            if feas is None:
                continue
            start, finish = feas
            travel = 0.0 if len(route) == 0 else inst.d(current_id, a.id)
            visited.add(a.id)
            route.append(a.id)
            dfs(a.id, finish, value + a.value, travel_cost + travel, visited, route)
            route.pop()
            visited.remove(a.id)
    if inst.start_id is None:
        for a in inst.activities:
            feas = feasible_transition(inst, a.id, a.id, 0.0)
            if feas is None:
                continue
            start, finish = feas
            dfs(a.id, finish, a.value, 0.0, {a.id}, [a.id])
    else:
        dfs(inst.start_id, 0.0, 0.0, 0.0, set(), [])
    if best is None:
        return State(priority=0.0, current_id=-1, time=0.0, collected_value=0.0, travel_cost=0.0,
                     visited=frozenset(), route=())
    return best



## 6) Experimentos


In [ ]:

def example_instance() -> Instance:
    acts = [
        Activity(0, "Gimnasio",   value=8, duration=1.0, open_time=7.0,  close_time=11.0),
        Activity(1, "Biblioteca", value=10, duration=2.0, open_time=9.0,  close_time=18.0),
        Activity(2, "Clases",     value=12, duration=2.0, open_time=8.0,  close_time=12.0),
        Activity(3, "Comedor",    value=5, duration=1.0, open_time=12.0, close_time=15.0),
        Activity(4, "Super",      value=6, duration=0.5, open_time=10.0, close_time=20.0),
    ]
    d = [
        [0,  0.5, 0.4, 0.6, 0.7],
        [0.5, 0,  0.7, 0.6, 0.4],
        [0.4, 0.7, 0,  0.5, 0.6],
        [0.6, 0.6, 0.5, 0,  0.3],
        [0.7, 0.4, 0.6, 0.3, 0  ],
    ]
    return Instance(activities=acts, travel_time=d, Tmax=19.0, alpha=0.2)

inst = example_instance()

print("Actividades:")
for a in inst.activities:
    print(" ", a)

print("\n--- A* ---")
best_astar = astar_maximize(inst)
print("Ruta:", [inst.activity_by_id(i).name for i in best_astar.route])
print("Tiempo final:", round(best_astar.time, 2))
print("Valor:", best_astar.collected_value, "Transporte:", round(best_astar.travel_cost, 2))
print("Objetivo:", round(best_astar.total_objective(inst.alpha), 2))

print("\n--- Branch & Bound ---")
best_bnb = branch_and_bound(inst)
print("Ruta:", [inst.activity_by_id(i).name for i in best_bnb.route])
print("Tiempo final:", round(best_bnb.time, 2))
print("Valor:", best_bnb.collected_value, "Transporte:", round(best_bnb.travel_cost, 2))
print("Objetivo:", round(best_bnb.total_objective(inst.alpha), 2))
